In [1]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh

from base64 import b64encode

In [2]:
%%time
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U pycocotools
!pip install -qr yolov5/requirements.txt  # install dependencies
!cp yolov5/requirements.txt .  # save requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16965, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (111/111), done.
Receiving objects: 100% (16965/16965), 15.71 MiB | 27.37 MiB/s, done.
remote: Total 16965 (delta 82), reused 99 (delta 49), pack-reused 16805 (from 1)
Resolving deltas: 100% (11615/11615), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
CPU times: user 97.9 ms, sys: 13 ms, total: 111 ms
Wall time: 10.9 s


In [3]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 992.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.9 MB/s eta 0:00:00


In [4]:
%pwd

'/content'

In [5]:
import yt_dlp

url = 'https://www.youtube.com/watch?v=NcaGFp76BTY'

ydl_opts = {'outtmpl': 'downloaded_video.mp4'}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=NcaGFp76BTY
[youtube] NcaGFp76BTY: Downloading webpage
[youtube] NcaGFp76BTY: Downloading ios player API JSON
[youtube] NcaGFp76BTY: Downloading web creator player API JSON
[youtube] NcaGFp76BTY: Downloading player d9418494
[youtube] NcaGFp76BTY: Downloading m3u8 information
[info] NcaGFp76BTY: Downloading 1 format(s): 136+251
[download] Destination: downloaded_video.mp4.f136.mp4
[download] 100% of    7.52MiB in 00:00:00 at 16.12MiB/s  
[download] Destination: downloaded_video.mp4.f251.webm
[download] 100% of   27.29KiB in 00:00:00 at 202.01KiB/s 
[Merger] Merging formats into "downloaded_video.mp4.mkv"
Deleting original file downloaded_video.mp4.f136.mp4 (pass -k to keep)
Deleting original file downloaded_video.mp4.f251.webm (pass -k to keep)


In [6]:
!ls

downloaded_video.mp4.mkv  requirements.txt  sample_data  yolov5


In [7]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [8]:
!ffmpeg -i downloaded_video.mp4.mkv -c copy downloaded_video.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [10]:
!ls

downloaded_video.mp4  downloaded_video.mp4.mkv	requirements.txt  sample_data  yolov5


In [11]:
!pwd

/content


In [12]:
!python yolov5/detect.py --source downloaded_video.mp4 --weights yolov5s.pt --conf 0.25

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['yolov5s.pt'], source=downloaded_video.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-369-g907bef2f Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100% 14.1M/14.1M [00:00<00:00, 131MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 pa

In [13]:
!ls

downloaded_video.mp4  downloaded_video.mp4.mkv	requirements.txt  sample_data  yolov5  yolov5s.pt


In [14]:
from google.colab import files

files.download('/content/yolov5/runs/detect/exp/downloaded_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import shutil

shutil.copy('/content/yolov5/runs/detect/exp/downloaded_video.mp4', '/content/drive/MyDrive/')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/'